# Web-scraping таблиц с Beautiful soup

Веб‑скрейпинг — это автоматизированный сбор данных сайтов. Парсинг — это часть веб‑скрейпинга, процесс приведения собранных данных в читаемый вид.

Начнем с простого примера, чтобы теперь разобраться, как из таблиц доставать информацию.

Сначала мы импортируем библиотеку `requests`. Она позволяет нам просто  и удобно посылать HTTP/1.1 запросы, не утруждаясь ручным трудом.

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
link = 'https://raw.githubusercontent.com/ifsytnik57/ad/main/simple_table.html?token=GHSAT0AAAAAACJWFWT2EB62XXFD5CX4X6TOZKBLSUQ'

Теперь мы должны указать адрес страницы с которой мы будем скрейпить данные и сохраним ее в переменную `page`.

In [ ]:
page = requests.get(link)

In [ ]:
page

<Response [200]>

Ответ
<Response [200]>

говорит об удачном завершении операции

Функция text передает html код страницы

In [ ]:
page.text

'<!DOCTYPE html>\n<html>\n<head>\n<title>Page Title</title>\n</head>\n<body>\n\n<h1>Моя первая html-страница</h1>\n<h2>О себе</h2>\n<p>Я учусь создавать html-страницы.</p>\n\n<table border="1">\n<caption>Информация</caption>\n<tr>\n<th>Фамилия</th>\n<th>Имя</th>\n<th>Возраст</th>\n</tr>\n<tr>\n<td>Тамбовцева</td>\n<td>Алла</td>\n<td>24</td>\n</tr>\n<tr>\n<td>Иванов</td>\n<td>Петр</td>\n<td>32</td>\n</tr>\n<tr>\n<td>Константинопольский</td>\n<td>Константин</td>\n<td>29</td>\n</tr>\n</table>\n    \n<table>\n<tr>\n<td>Тамбовцева</td>\n<td>Алла</td>\n<td>24</td>\n</tr>\n</table>\n\n</body>\n</html>'

Как мы видим, весь код представлен просто блоком текста, который неудобно читать и разбирать. Поэтому мы создадим объект `BeautifulSoup` с помощью функциии `BeautifulSoup`, предварительно импортировав саму библиотеку. `Beautiful Soup` это библиотека для парсинга `HTML` и `XML` документов. Она создает дерево из `HTML` кода, что очень полезно при скрейпинге. Функция `prettify()` позволяет видеть код в более удобном виде, в том числе с разбивкой по тегам.

In [ ]:
soup = BeautifulSoup(page.text)
print(soup.prettify())

Видим, что .preffify() упорядочил наш html код и представил его в видел дерева и вложенных узлов. Давайте попробуем найти таблицу. За поиск отвечает метод .find_all(), которому мы передаем тэг для поиска.

In [ ]:
tables = soup.find_all('table')
print(tables)

Получили на выходе список, который содержит все элементы, принадлежащие к тэгу. В этом примере таблиц у нас две, каждая из них является отдельным элементом списка.

In [ ]:
tables[1]

<table>
<tr>
<td>Тамбовцева</td>
<td>Алла</td>
<td>24</td>
</tr>
</table>

Иногда таблиц очень много (уже скоро увидим такое на реальном примере), поэтому поисковый запрос можно уточнить с помощью аттрибутов тэга. Так, если посмотрим на код, увидим, что у нашей первой таблицы есть атрибут border со значением 1. Передадим это уточнение в bs в виде словаря. То, что до знака равно (атрибут) - ключ, а его значение - значение.

In [ ]:
tables = soup.find_all('table', {'border':1}) # так поиск вернет только одну таблицу.
print(tables)

[<table border="1">
<caption>Информация</caption>
<tr>
<th>Фамилия</th>
<th>Имя</th>
<th>Возраст</th>
</tr>
<tr>
<td>Тамбовцева</td>
<td>Алла</td>
<td>24</td>
</tr>
<tr>
<td>Иванов</td>
<td>Петр</td>
<td>32</td>
</tr>
<tr>
<td>Константинопольский</td>
<td>Константин</td>
<td>29</td>
</tr>
</table>]


Давайте теперь, например, в один список сохраним фамилии людей из нашей таблицы, а в другой возраст.

In [ ]:
rows = tables[0].find_all('tr') # нашли теперь все ряды в таблице
print(rows[0])

<tr>
<th>Фамилия</th>
<th>Имя</th>
<th>Возраст</th>
</tr>


Нашли все ряды. Первый ряд с заголовками колонок (тэги th, можно его пропустить).

In [ ]:
rows[1].find_all('td')

[<td>Тамбовцева</td>, <td>Алла</td>, <td>24</td>]

Добрались до внутреннего тэга td - глубже идти некуда. Тут три элемента (три колонки). Давайте забирать отсюда возраст и фамилию.

In [ ]:
rows[1].find_all('td')[0].text.strip()

'Тамбовцева'

Чтобы избавиться от тэгов используем атрибут text - выводит текст, который лежит между тэгами. Еще для надежности можно использовать strip() - удалит невидимые символы, если они есть, а если их нет, то ошибки тоже не выдаст.

In [ ]:
surnames = []
ages = []

for row in rows[1:]: # начинаем цикл со второго объекта
    surnames.append(row.find_all('td')[0].text.strip())
    ages.append(row.find_all('td')[2].text.strip())

print(surnames, ages)

['Тамбовцева', 'Иванов', 'Константинопольский'] ['24', '32', '29']


А теперь давайте запишем собранную информацию в файл. Табличные данные часто хранятся в формате csv - comma separated values, поэтому будем просто записывать значения в текстовый файл через запятую.

In [ ]:
f = open('table.csv', 'w', encoding = 'utf8')

for idx in range(len(surnames)):
    f.write(f'{surnames[idx]},{ages[idx]}\n')

f.close()

In [ ]:
with open('table.csv', encoding='utf8') as f:
    for line in f:
        print(line)

Тамбовцева,24

Иванов,32

Константинопольский,29



посмотрим, как выглядят  в таблице pandas только что собранные данные

In [ ]:
import pandas as pd
df = pd.read_csv('table.csv', encoding='utf8', header=None)
df.head()

,0,1
0,Тамбовцева,24
1,Иванов,32
2,Константинопольский,29


# парсим реальную табличку с wiki

Будем работать со страницей вики, на которой находится информация о испытаниях ядерного оружия.
https://en.wikipedia.org/wiki/List_of_nuclear_weapons_tests

Нашей задачей будет достать вторую таблицу Worldwide nuclear test with a yield of 1.4 Mt TNT equivalent and more и сохранить из нее в три списка три колонки - Date, Yield (мощность взрыва) и Country.

In [ ]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_nuclear_weapons_tests').text
# print(website_url)

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(website_url,'lxml')

# print(soup.prettify())

Здесь таблиц уже будет много, и чтобы найти нужную уже точно понадобится атрибут.

In [ ]:
len(soup.find_all('table'))

19

Если внимательно изучить код `HTML` искомой таблицы, то можно обнаружить что вся таблица находится в классе `Wikitable Sortable` (для включения отображения кода сайта в вашем браузере можно нажать правкой кнопкой мыши на таблицу и выбрать пункт *Исследовать элемент* или в любом месте страницы и *Показать исходный код*).

Поэтому первой задачей будет найти класс *wikitable sortable* в коде `HTML`. Это можно сделать с помощью функции `find_all`, указав в качестве аргументов, что мы ищем тэг `table` с классом `wikitable sortable`.

In [ ]:
My_table = soup.find_all('table',{'class':'wikitable sortable'})
# My_table

Но как вы могли заметить, то на страницы есть две таблицы, которые принадлежат этому классу. Функция `find_all` вернет все найденные объекты в виде списка. Поэтому проверим второй найденный элемент.

In [ ]:
My_table[1]

Все верно, это наша искомая таблица. Если дальше изучить содержимое таблицы, то станет понятно что внутри тега `th` находится заголовок таблицы, а внутри `td` строки таблицы. А оба этих тега находятся внутри тегов `tr` что является по факту строкой таблицы. Давайте извлечем все строки таблицы также используя функцию `find_all`.

In [ ]:
rows = My_table[1].find_all('tr')
# print(rows)

Давайте внимательно изучим содержимое одной строки, вытащим все `td`. Отобразим вторую строчку:

In [ ]:
rows[1].find_all('td')

[<td>October 30, 1961</td>,
 <td>50</td>,
 <td>parachute air drop</td>,
 <td>Soviet Union</td>,
 <td><a href="/wiki/Novaya_Zemlya" title="Novaya Zemlya">Novaya Zemlya</a></td>,
 <td><a href="/wiki/Tsar_Bomba" title="Tsar Bomba">Tsar Bomba</a>, Test #130
 </td>]

Мы видим нужные нам данные между тегов `<td><\td>`, а также ссылки с тегом `<a>` и даже смешанные ячейки с обоими этими вариантами. Давайте сначала извлечем просто данные. Для этого используем функцию `get_text()` - она вернет все что между тегами.

Возьмем, например, дату (она будет первым элементом):

In [ ]:
rows[1].find_all('td')[0].get_text()

'October 30, 1961'

Единственное, нам нужно отдельно обработать, это первую строку, в которой хранится заголовок ряда (table header)

In [ ]:
rows[0].find_all('th')[0].get_text()

'Date (GMT)\n'

Все классно, только давайте избавимся от знака переноса строки.

In [ ]:
rows[0].find_all('th')[0].get_text().strip()

'Date (GMT)'

Давайте теперь извлечем все даты. Создадим список для их хранения `Dates` и будет итерироваться по всем элементам:

In [ ]:
Dates = []

Dates.append(rows[0].find_all('th')[0].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    Dates.append(r[0].get_text().strip()) # сохраняем данные в наш список

print(Dates)

['Date (GMT)', 'October 30, 1961', 'December 24, 1962', 'August 5, 1962', 'September 27, 1962', 'September 25, 1962', 'March 1, 1954', 'May 5, 1954', 'October 23, 1961', 'March 26, 1954', 'October 31, 1952', 'August 25, 1962', 'September 19, 1962', 'July 11, 1958', 'June 28, 1958', 'October 30, 1962', 'October 22, 1962', 'June 27, 1962', 'April 25, 1954', 'July 20, 1956', 'October 31, 1961', 'November 6, 1971', 'July 10, 1956', 'August 27, 1962', 'October 6, 1961', 'October 27, 1973', 'November 17, 1976', 'July 11, 1962', 'May 20, 1956', 'August 1, 1958', 'August 12, 1958', 'September 12, 1973', 'May 27, 1956', 'October 14, 1970', 'September 16, 1962', 'June 17, 1967', 'September 15, 1962', 'December 25, 1962', 'April 28, 1958', 'October 4, 1961', 'June 10, 1962', 'December 27, 1968', 'September 29, 1969', 'June 27, 1973', 'October 6, 1957', 'October 18, 1958', 'October 22, 1958', 'August 20, 1962', 'September 10, 1961', 'August 24, 1968', 'September 27, 1971', 'September 21, 1962', 'N

Ок! Следующие колонки, которые нам нужны - мощность взрыва и страна. Давайте поймем, где их искать.

In [ ]:
rows[0]

<tr>
<th>Date (GMT)
</th>
<th>Yield (megatons)
</th>
<th>Deployment
</th>
<th>Country
</th>
<th>Test Site
</th>
<th>Name or Number
</th></tr>

Видим, что Yield вторая колонка, а страна третья. Соберем их в отдельные списки по той же схеме, что дату. Но сначала проверим, что правильно посчитали номера.

In [ ]:
rows[0].find_all('th')[1]

<th>Yield (megatons)
</th>

In [ ]:
rows[0].find_all('th')[3]

<th>Country
</th>

Вроде все правильно. Единственно, не забудем хранить числа как float.

In [ ]:
Yield = []

Yield.append(rows[0].find_all('th')[1].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    Yield.append(float(r[1].get_text().strip())) # сохраняем данные в наш список и переводим в float

print(Yield)

['Yield (megatons)', 50.0, 24.2, 21.1, 20.0, 19.1, 15.0, 13.5, 12.5, 11.0, 10.4, 10.0, 10.0, 9.3, 8.9, 8.3, 8.2, 7.7, 6.9, 5.0, 5.0, 4.8, 4.5, 4.2, 4.0, 4.0, 4.0, 3.9, 3.8, 3.8, 3.8, 3.8, 3.5, 3.4, 3.3, 3.3, 3.1, 3.1, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.9, 2.9, 2.8, 2.8, 2.7, 2.6, 2.5, 2.4, 2.3, 2.2, 2.0, 1.9, 1.9, 1.9, 1.8, 1.7, 1.6, 1.6, 1.6, 1.6, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.4, 1.4, 1.4, 1.4]


In [ ]:
Country = []

Country.append(rows[0].find_all('th')[3].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    Country.append(r[3].get_text().strip()) # сохраняем данные в наш список и переводим в float

print(Country)

['Country', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'USA', 'USA', 'Soviet Union', 'USA', 'USA', 'Soviet Union', 'Soviet Union', 'USA', 'USA', 'USA', 'Soviet Union', 'USA', 'USA', 'USA', 'Soviet Union', 'USA', 'USA', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'China', 'USA', 'USA', 'USA', 'USA', 'Soviet Union', 'USA', 'China', 'Soviet Union', 'China', 'Soviet Union', 'Soviet Union', 'UK', 'Soviet Union', 'USA', 'China', 'China', 'China', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'France', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'USA', 'USA', 'Soviet Union', 'USA', 'UK', 'USA', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'USA', 'Soviet Union', 'USA', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'Soviet Union', 'USA', 'USA', 'USA', 'Soviet Union']


In [ ]:
print(Dates)
print(Yield)
print(Country)

['Date (GMT)', 'October 30, 1961', 'December 24, 1962', 'August 5, 1962', 'September 27, 1962', 'September 25, 1962', 'March 1, 1954', 'May 5, 1954', 'October 23, 1961', 'March 26, 1954', 'October 31, 1952', 'August 25, 1962', 'September 19, 1962', 'July 11, 1958', 'June 28, 1958', 'October 30, 1962', 'October 22, 1962', 'June 27, 1962', 'April 25, 1954', 'July 20, 1956', 'October 31, 1961', 'November 6, 1971', 'July 10, 1956', 'August 27, 1962', 'October 6, 1961', 'October 27, 1973', 'November 17, 1976', 'July 11, 1962', 'May 20, 1956', 'August 1, 1958', 'August 12, 1958', 'September 12, 1973', 'May 27, 1956', 'October 14, 1970', 'September 16, 1962', 'June 17, 1967', 'September 15, 1962', 'December 25, 1962', 'April 28, 1958', 'October 4, 1961', 'June 10, 1962', 'December 27, 1968', 'September 29, 1969', 'June 27, 1973', 'October 6, 1957', 'October 18, 1958', 'October 22, 1958', 'August 20, 1962', 'September 10, 1961', 'August 24, 1968', 'September 27, 1971', 'September 21, 1962', 'N

Давайте опять сохраним данные в файл. Тут обратите внимание на запятую в дате - давайте сделаем разделителем табуляцию.

In [ ]:
f = open('nuclear.csv', 'w')

for idx in range(len(Dates)):
    f.write(f'{Dates[idx]}\t{Yield[idx]}\t{Country[idx]}\n')

f.close()

In [ ]:
df = pd.read_csv('nuclear.csv', sep='\t', index_col=None)
df.head()

,Date (GMT),Yield (megatons),Country
0,"October 30, 1961",50.0,Soviet Union
1,"December 24, 1962",24.2,Soviet Union
2,"August 5, 1962",21.1,Soviet Union
3,"September 27, 1962",20.0,Soviet Union
4,"September 25, 1962",19.1,Soviet Union


## Задание 1

Необходимо по вариантам из таблиц хода подачи заявлений абитуриентов с сайта СГТУ

https://abitur.sstu.ru/vpo/level/2023/b/o

https://abitur.sstu.ru/vpo/direction/2023/18/b/o/b

 достать следующую информацию:

 Сумма баллов, 	ПММ / МАТ,	ПФМ / ПИТ / ИНФ / ФИЗ,	РУС,	Сумма баллов
по ИД (все),	Приоритет, Состояние.

7 колонок сохранить в файле csv

**Варианты:**

1. Направление ИВЧТ, колонка Бюджет/Списки, таблица "Прием на целевое обучение
Роскосмос", таблица "Прием на целевое обучение"

2. Направление ИВЧТ, колонка Бюджет/Списки, таблица "Имеющие особые права"

3. Направление ИВЧТ, колонка Бюджет/Списки, таблица "По отдельной квоте"

4.Направление ИВЧТ, колонка Бюджет/Списки, таблица "По общему конкурсу"

5.Направление ИВЧТ, колонка По договорам/Списки, таблица "На платной основе"

6. Направление ПИНЖ, колонка Бюджет/Списки, таблица "Прием на целевое обучение
Электроприбор", таблица "Прием на целевое обучение СергоОрджоникидзе", таблица "Прием на целевое обучение СЭПО"

7. Направление ПИНЖ, колонка Бюджет/Списки, таблица "Имеющие особые права"

8. Направление ПИНЖ, колонка Бюджет/Списки, таблица "По отдельной квоте"

9.Направление ПИНЖ, колонка Бюджет/Списки, таблица "По общему конкурсу"

10.Направление ПИНЖ, колонка По договорам/Списки, таблица "На платной основе"

11. Направление ИФСТ, колонка Бюджет/Списки, таблица "Прием на целевое обучение
СигналГлухарева",  таблица "Прием на целевое обучение СергоОрджоникидзе", таблица "рием на целевое обучение СЭПО"

12. Направление ИФСТ, колонка Бюджет/Списки, таблица "Прием на целевое обучение Роскосмос", таблица "Прием на целевое обучение"

13. Направление ИФСТ, колонка Бюджет/Списки, таблица "Имеющие особые права"

14. Направление ИФСТ, колонка Бюджет/Списки, таблица "По отдельной квоте"

15.Направление ИФСТ, колонка Бюджет/Списки, таблица "По общему конкурсу"

16.Направление ИФСТ, колонка По договорам/Списки, таблица "На платной основе"

17. Направление ПИНФ, колонка Бюджет/Списки, таблица "Прием на целевое обучение
СигналГлухарева",  таблица "Прием на целевое обучение"

18. Направление ИФСТ, колонка Бюджет/Списки, таблица "Имеющие особые права"

19. Направление ИФСТ, колонка Бюджет/Списки, таблица "По отдельной квоте"

20.Направление ИФСТ, колонка Бюджет/Списки, таблица "По общему конкурсу"

21.Направление ИФСТ, колонка По договорам/Списки, таблица "На платной основе"




